In [21]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from cleaning_data import *
from stats import *
from functions import *
from clean_data_testing import *
from utils import split_data, downsample_class_0, upsample_class_1_to_percentage
import datetime
import seaborn as sns
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from functions import *

In [23]:
from clean_data_testing import *

In [24]:
DATA_PATH = '/Users/williamjallot/Desktop/ML/dataset'
x_train, x_test, y_train, train_ids, test_ids, labels =  load_csv_data(DATA_PATH, sub_sample=False)
labels.pop(0) 

'Id'

In [25]:
#Split the data into training and testing
x_tr, x_te, y_tr, y_te = split_data(x_train, y_train, 0.8, seed= 2)

In [26]:
#For simplicty, replace the label -1 to 0
y_tr[y_tr == -1] = 0

In [27]:
#Up sample the data
x_tr_up, y_tr_up = upsample_class_1_to_percentage(x_tr, y_tr, 0.3)

In [54]:
#Down sample the data
x_tr_ds, y_tr_ds = downsample_class_0(x_tr, y_tr, down_sampling_size)

In [29]:

x_train_cleaned, features, median_and_most_probable_class, W, mean = clean_train_data(x_tr_up, labels)

Number of components to retain 90.0% variance: 63


In [30]:
x_te_cleaned = clean_test_data(x_te, labels, features, median_and_most_probable_class, mean, W)

In [31]:
x_test_cleaned = clean_test_data(x_test, labels, features, median_and_most_probable_class, mean, W)

In [32]:
x_te_cleaned.shape

(65627, 63)

In [33]:
x_train_cleaned.shape

(342081, 63)

In [34]:
max_iters = 100000
gamma = 0.001
batch_size = 64
lambda_ =  0.005
num_samples = x_train_cleaned.shape[0]



# Parameters for the Gaussian distribution
mean = 0    # Mean of the distribution
std_dev = 1 # Standard deviation of the distribution

tx = np.c_[np.ones(num_samples), x_train_cleaned]
w_initial = np.random.normal(loc=mean, scale=std_dev, size=tx.shape[1])


sgd_losses, sgd_ws = stochastic_gradient_descent(
y_tr_up, tx, w_initial, batch_size, max_iters, gamma, lambda_
)


In [35]:
num_samples = x_te.shape[0]
tx_te = np.c_[np.ones(num_samples), x_te_cleaned]
y_predict = predict(tx_te, sgd_ws[-1])

In [36]:
accuracy(y_te, y_predict)

0.801057491581209

In [37]:
f1_score(y_te, y_predict)

(0.38181818181818183, 0.6806212018906145)

In [38]:
num_samples = x_test_cleaned.shape[0]
tx_test = np.c_[np.ones(num_samples), x_test_cleaned]
y_test_to_save = predict(tx_test, sgd_ws[-1])

In [39]:
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/sample_submission.csv", y_test_to_save, delimiter=",")

In [42]:
# Stack the ids and predictions together column-wise
submit = np.column_stack((test_ids, y_test_to_save))

# Save to a CSV file using np.savetxt
np.savetxt("/Users/williamjallot/Desktop/ML/dataset/sample_submission.csv", submit, delimiter=",", fmt='%d,%d', header='Id,Prediction', comments='')